# Imports

In [2]:
import pandas as pd
import os

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import shap

import ollama

In [3]:
# CSV file names
base_data_prep_name = "tabular_data_preprocessed_2025_04_03.csv"

In [4]:
# Force CUDA usage
os.environ["OLLAMA_BACKEND"] = "cuda"
os.environ["OLLAMA_NUM_THREADS"] = "16"

# Load Data

In [6]:
# Load data
data = pd.read_csv(base_data_prep_name)
X = data.drop(columns=["income"])
y = data["income"]                

# Demonstration LLM explainability / interpreability

### Set up train test set and learn models 

In [9]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train models
lr = LogisticRegression().fit(X_train, y_train)
rf = RandomForestClassifier().fit(X_train, y_train)

### Model performance

In [11]:
print("LR Accuracy:", accuracy_score(y_test, lr.predict(X_test)))
print("RF Accuracy:", accuracy_score(y_test, rf.predict(X_test)))

LR Accuracy: 0.8417442931722796
RF Accuracy: 0.8485003582761798


## Extract model weights

In [13]:
# Logistic Regression coefficients
lr_weights = pd.DataFrame({
    "Feature": X.columns,
    "Weight": lr.coef_[0]
}).sort_values("Weight", ascending=False)

# Random Forest importances
rf_importances = pd.DataFrame({
    "Feature": X.columns,
    "Importance": rf.feature_importances_
}).sort_values("Importance", ascending=False)

## Explain model

In [15]:
# Explain Logistic Regression weights
lr_prompt = f"""
Explain the following logistic regression weights for predicting income: 
{lr_weights.to_string()}.
Highlight the top 5 most impactful features and their likely relationship with income.
"""

print("Prompt:")
print(lr_prompt)
print()

lr_explanation = ollama.generate(model="mistral",prompt=lr_prompt)['response']
print("Explanation")
print(lr_explanation)

Prompt:

Explain the following logistic regression weights for predicting income: 
                                 Feature    Weight
4                           capital_gain  2.321662
14     marital_status_married_civ_spouse  1.163453
2                          education_num  0.907675
6                         hours_per_week  0.364210
23                              sex_male  0.329260
0                                    age  0.309664
5                           capital_loss  0.271564
8             relationship_not_in_family  0.261162
12                     relationship_wife  0.225362
22                            race_white  0.164979
11                relationship_unmarried  0.124523
13      marital_status_married_af_spouse  0.071284
19               race_asian_pac_islander  0.066926
20                            race_black  0.062537
3                             occupation  0.058830
7                         native_country  0.044228
18                marital_status_widowed  0.013017

In [16]:
# Explain Random Forest importances
rf_prompt = f"""
Explain these Random Forest feature importances for income prediction:
{rf_importances.to_string()}.
Compare the top 5 features to the logistic regression results.
"""

print("Prompt:")
print(rf_prompt)
print()

rf_explanation = ollama.generate(model="mistral",prompt=rf_prompt)['response']
print("Explanation:")
print(rf_explanation)

Prompt:

Explain these Random Forest feature importances for income prediction:
                                 Feature  Importance
0                                    age    0.218619
2                          education_num    0.136656
4                           capital_gain    0.118766
6                         hours_per_week    0.110285
14     marital_status_married_civ_spouse    0.108251
3                             occupation    0.082486
1                              workclass    0.047641
5                           capital_loss    0.041683
16          marital_status_never_married    0.035012
7                         native_country    0.020166
23                              sex_male    0.017234
8             relationship_not_in_family    0.016482
12                     relationship_wife    0.008734
10                relationship_own_child    0.008513
11                relationship_unmarried    0.007261
22                            race_white    0.005595
20                 

# Explaing individual rows 
We use shapley values as well as class prediction probabilities to provide a natural explanation

### Our sample with with its values, label and shapley values

In [19]:
# Initialize SHAP explainer
explainer_lr = shap.LinearExplainer(lr, X_train)

# Sample
sample = X_test.iloc[0:1] 
sample_target = y_test.iloc[0:1] 
display(sample)
print()
print("Target:")
display(sample_target)

# Get SHAP values
shap_values_lr = explainer_lr.shap_values(sample)
print("\nShapley values:")
print(shap_values_lr)

,age,workclass,education_num,occupation,capital_gain,capital_loss,hours_per_week,native_country,relationship_not_in_family,relationship_other_relative,...,marital_status_married_civ_spouse,marital_status_married_spouse_absent,marital_status_never_married,marital_status_separated,marital_status_widowed,race_asian_pac_islander,race_black,race_other,race_white,sex_male
7762,-1.505691,0.088484,-0.419335,0.336204,-0.144804,-0.217127,-1.64812,0.289462,1.697524,-0.178368,...,-0.919604,-0.114128,1.424944,-0.179829,-0.1791,-0.179161,-0.325728,-0.091554,0.411743,0.70422



Target:


7762    0
Name: income, dtype: int64


Shapley values:
[[-4.75664132e-01 -1.46733139e-03 -3.10684818e-01  2.18329447e-02
  -4.39785804e-02 -4.98517962e-02 -5.96370295e-01  1.36520322e-02
   4.47883632e-01  1.45049350e-02  1.12897068e-01 -2.03162052e-02
  -7.39986118e-02 -2.47315504e-19 -1.12084008e+00  3.49971541e-04
  -2.76121484e-01  6.49514621e-03  3.61297052e-19 -1.15663668e-02
  -1.27416633e-02 -1.19126690e-03  5.15475497e-02  1.95838589e-01]]


### Class prediction deails 

In [21]:
# Get prediction details
pred_prob = lr.predict_proba(sample)[0][0] # Class 0 (earning less than 50k dollars per year)
pred_class = lr.predict(sample)[0]

print("Predicted class: " + str(pred_class))
print("Model probability: " + str(round(pred_prob * 100)) + "%")

Predicted class: 0
Model probability: 99%


### NLG Preperation

In [23]:
# Get feature names and SHAP values
feature_names = X_train.columns.tolist()
shap_values = shap_values_lr[0]

# Create a list of (feature_name, shap_value) pairs and sort by absolute value
feature_importance = sorted(zip(feature_names, shap_values), key=lambda x: abs(x[1]), reverse=True)

# Generate the prompt
prompt = f"""
I'm analyzing a machine learning model's prediction using SHAP values.
Explain why the logistic regression model predicted income class = {pred_class} 
(probability: {pred_prob:.2f}) for this individual:

Individual details:
- Features: {sample.to_dict('records')[0]}
- Actual target value: {sample_target.values[0]}

The top 10 features influencing this prediction, based on Shapley values, are:
"""

# Add each feature's contribution
for i, (feature, value) in enumerate(feature_importance[:10], 1):
    direction = "increased" if value > 0 else "decreased"
    prompt += f"{i}. {feature}: {value:.4f} (this feature {direction} the prediction)\n"

In [24]:
print(prompt)


I'm analyzing a machine learning model's prediction using SHAP values.
Explain why the logistic regression model predicted income class = 0 
(probability: 0.99) for this individual:

Individual details:
- Features: {'age': -1.5056913908365146, 'workclass': 0.088484476319535, 'education_num': -0.4193352748208159, 'occupation': 0.33620420122783, 'capital_gain': -0.144803531037397, 'capital_loss': -0.2171270991958307, 'hours_per_week': -1.6481203808733793, 'native_country': 0.2894622051340574, 'relationship_not_in_family': 1.697523568963296, 'relationship_other_relative': -0.1783679035251107, 'relationship_own_child': -0.4286406685819893, 'relationship_unmarried': -0.3423905416697579, 'relationship_wife': -0.2238686592220623, 'marital_status_married_af_spouse': -0.0275339616688154, 'marital_status_married_civ_spouse': -0.9196038900884226, 'marital_status_married_spouse_absent': -0.1141282715716187, 'marital_status_never_married': 1.424943762639813, 'marital_status_separated': -0.17982913

### Prompt LLM to explain the prediction

In [26]:
sample_explanation = ollama.generate(model="mistral", prompt=prompt)['response']

print("SHAP Explanation for Logistic Regression:")
print(sample_explanation)

SHAP Explanation for Logistic Regression:
 The logistic regression model predicted income class = 0 (probability: 0.99) for this individual based on the input features and the coefficients learned during training. To understand why this prediction was made, we can focus on the top 10 Shapley values that influenced this prediction.

The top five features that decreased the prediction are 'marital_status_married_civ_spouse', 'hours_per_week', 'age', 'education_num', and 'marital_status_never_married'. These features had negative Shapley values, which means they contributed negatively to the prediction.

1. Marital status married civil spouse (marital_status_married_civ_spouse) had the most significant impact with a Shapley value of -1.1208. Being married to a civilian spouse decreased the predicted probability of income class = 0, as this feature negatively influenced the prediction.

2. Hours per week (hours_per_week) also contributed negatively to the prediction with a Shapley value of